# Feature selection
Notebook 01 showed how to select the top 100
but it did not use describe().

Here, use describe() which takes much longer.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-05-26 12:51:09.391000
Python 3.8.3
sklearn 1.1.1


[]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
import joblib # used to dump/load sklearn models

In [3]:
# Data straight from CellProfiler (rows for every patch image file)
BASE_DIR='//Users/jasonmiller/Downloads/Process100/'
CLASS_DIRS=['0','1','2','3','4','5']
NUM_CLASSES=len(CLASS_DIRS)
IMG_FN="Process100_Image.csv"
RBC_FN="Process100_MergeRBC.csv"
NUC_FN="Process100_Nucleus.csv"
CEL_FN="Process100_ExpandCells.csv"
TIS_FN="Process100_Tissue.csv"
# Saving our models
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.01/'
DESCRIBE=False  # just retain the mean over objects in patch (not even the count!)
DESCRIBE=True   # compute stats for every column
SHOW_ALL_COLUMNS=True   # show all the csv columns in this notebook  
SHOW_ALL_COLUMNS=False  
SHOW_DEMO = False  # show one class before loading all classes

In [4]:
if SHOW_ALL_COLUMNS:
    nuc_df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN)
    initial_cols = nuc_df.columns
    for col in initial_cols:
        print(col)
    nuc_df = None

In [5]:
if SHOW_ALL_COLUMNS:
    rbc_df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+RBC_FN)
    initial_cols = rbc_df.columns
    for col in initial_cols:
        print(col)
    rbc_df = None

In [6]:
def get_nuc_columns():
    df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN)
    initial_cols = df.columns
    GOOD_COLS=['ImageNumber']
    GOOD_COLS.extend(['AreaShape_Area','AreaShape_Compactness','AreaShape_ConvexArea'])
    GOOD_COLS.extend(['AreaShape_Eccentricity','AreaShape_EquivalentDiameter','AreaShape_EulerNumber'])
    GOOD_COLS.extend(['AreaShape_Extent','AreaShape_FormFactor','AreaShape_MajorAxisLength'])
    GOOD_COLS.extend(['AreaShape_MaxFeretDiameter','AreaShape_MaximumRadius','AreaShape_MeanRadius'])
    GOOD_COLS.extend(['AreaShape_MinFeretDiameter','AreaShape_MinorAxisLength','AreaShape_Orientation'])
    GOOD_COLS.extend(['AreaShape_Perimeter','AreaShape_Solidity'])
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_CentralMoment') ) ] )
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_HuMoment') ) ] )
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_NormalizedMoment') ) ] )
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_SpacialMoment') ) ] )  # yes
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_Zernike') ) ] )   # yes
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Granularity') ) ] )   # yes!!!
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Intensity') ) ] )   # yes!!!
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Neighbors') ) ] )   
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('RadialDistribution') ) ] )   
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_') ) ] )  # yes
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_Contrast_Hematoxylin') ) ] )   
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_DifferenceEntropy_Hematoxylin') ) ] ) 
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_Entropy_Hematoxylin') ) ] )  # yes
    return GOOD_COLS
def get_rbc_columns():
    df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+RBC_FN)
    initial_cols = df.columns
    GOOD_COLS=['ImageNumber']
    GOOD_COLS.extend(['AreaShape_Area','AreaShape_Compactness','AreaShape_ConvexArea'])
    GOOD_COLS.extend(['AreaShape_Eccentricity','AreaShape_EquivalentDiameter','AreaShape_EulerNumber'])
    GOOD_COLS.extend(['AreaShape_Extent','AreaShape_FormFactor','AreaShape_MajorAxisLength'])
    GOOD_COLS.extend(['AreaShape_MaxFeretDiameter','AreaShape_MaximumRadius','AreaShape_MeanRadius'])
    GOOD_COLS.extend(['AreaShape_MinFeretDiameter','AreaShape_MinorAxisLength','AreaShape_Orientation'])
    GOOD_COLS.extend(['AreaShape_Perimeter','AreaShape_Solidity'])
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_CentralMoment') ) ] )
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_HuMoment') ) ] )
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_NormalizedMoment') ) ] )
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_SpacialMoment') ) ] )  
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_Zernike') ) ] )  
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Granularity') ) ] )  
    GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Intensity') ) ] )   
    return GOOD_COLS

In [7]:
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_one_file(fn,cols,prefix):
    set_prefix(prefix)
    object_df = pd.read_csv(fn)
    object_df = object_df[cols]
    object_df.rename(get_prefix,axis='columns',inplace=True)
    object_df.rename(columns={get_prefix('ImageNumber'):'ImageNumber'},inplace=True)
    if DESCRIBE:
        patch_df = object_df.groupby(['ImageNumber']).describe() ## this is slow
        # The random forest classifier cannot handle two-part column names, so convert them to string.    
        #patch_df.columns=patch_df.columns.map(str) # works but leaves in parenthesis
        patch_df.columns=patch_df.columns.map('_'.join)
    else:
        # Problem: using this approach, we don't get a cell count.
        #patch_df = object_df.groupby(['ImageNumber']).mean()
        patch_df = object_df.groupby(['ImageNumber']).max()
    # By virtue of groupby, ImageNumber is now the dataframe index.
    return patch_df


In [8]:
print(datetime.datetime.now())
total_df = None
if SHOW_DEMO:
    nuc_cols = get_nuc_columns()
    rbc_cols = get_rbc_columns()
    nuc_df = load_one_file(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN,nuc_cols,'Nuc_')
    rbc_df = load_one_file(BASE_DIR+CLASS_DIRS[0]+'/'+RBC_FN,rbc_cols,'Rbc_')
    total_df = nuc_df.join(rbc_df,how='outer')
    # Sources of NaN:
    # CellProfiler reports NaN for stdev of 1 value.
    # CellProfiler omits patches with no RBC; pandas join sets those to NaN.
    total_df.fillna(0,inplace=True)  
    print(datetime.datetime.now())
total_df

2022-05-26 12:51:15.560594


In [9]:
print(datetime.datetime.now())
df = None
def load_all_classes():
    nuc_cols = get_nuc_columns()
    rbc_cols = get_rbc_columns()    
    nuc_df = load_one_file(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN,nuc_cols,'Nuc_')
    rbc_df = load_one_file(BASE_DIR+CLASS_DIRS[0]+'/'+RBC_FN,rbc_cols,'Rbc_')
    X = nuc_df.join(rbc_df,how='outer')
    size = len(X)
    y = np.zeros(size)
    for i in range(1,NUM_CLASSES):
        nuc_df = load_one_file(BASE_DIR+CLASS_DIRS[i]+'/'+NUC_FN,nuc_cols,'Nuc_')
        rbc_df = load_one_file(BASE_DIR+CLASS_DIRS[i]+'/'+RBC_FN,rbc_cols,'Rbc_')
        Xi = nuc_df.join(rbc_df,how='outer')
        size = len(Xi)
        yi = np.ones(size) * i
        X = pd.concat( (X,Xi) )
        y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
X,y=load_all_classes()
X

2022-05-26 12:51:15.584182


,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_Compactness_count,Nuc_AreaShape_Compactness_mean,...,Rbc_Intensity_StdIntensity_Eosin_75%,Rbc_Intensity_StdIntensity_Eosin_max,Rbc_Intensity_UpperQuartileIntensity_Eosin_count,Rbc_Intensity_UpperQuartileIntensity_Eosin_mean,Rbc_Intensity_UpperQuartileIntensity_Eosin_std,Rbc_Intensity_UpperQuartileIntensity_Eosin_min,Rbc_Intensity_UpperQuartileIntensity_Eosin_25%,Rbc_Intensity_UpperQuartileIntensity_Eosin_50%,Rbc_Intensity_UpperQuartileIntensity_Eosin_75%,Rbc_Intensity_UpperQuartileIntensity_Eosin_max
ImageNumber,,,,,,,,,,,,,,,,,,,,,
1,1.0,313.000000,0.000000,313.0,313.00,313.0,313.00,313.0,1.0,1.273323,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3.0,251.000000,79.680612,178.0,208.50,239.0,287.50,336.0,3.0,1.816912,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,5.0,302.000000,162.260593,182.0,212.00,260.0,271.00,585.0,5.0,1.180711,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,29.0,314.137931,125.210030,177.0,233.00,262.0,340.00,635.0,29.0,1.366962,...,0.063568,0.065724,4.0,0.796560,0.021276,0.773728,0.781025,0.797439,0.812975,0.817634
5,32.0,355.937500,160.779339,180.0,243.00,331.0,375.25,832.0,32.0,1.353498,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,47.0,574.255319,290.847448,177.0,305.50,534.0,746.00,1323.0,47.0,1.459147,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
97,5.0,667.600000,270.346629,280.0,513.00,734.0,888.00,923.0,5.0,1.697802,...,0.025481,0.025481,1.0,0.760939,0.000000,0.760939,0.760939,0.760939,0.760939,0.760939
98,50.0,549.580000,293.608798,181.0,347.00,440.0,728.75,1565.0,50.0,1.360029,...,0.027539,0.027539,1.0,0.758598,0.000000,0.758598,0.758598,0.758598,0.758598,0.758598


In [10]:
print(datetime.datetime.now())
Xtrain,Xvalid,ytrain,yvalid = train_test_split(X, y.ravel()) 
        # ,random_state=42) # add this for reproducibility
print(datetime.datetime.now())
print('Xtrain',Xtrain.shape,'ytrain',ytrain.shape,'non-zero:',np.count_nonzero(ytrain))
print('Xvalid',Xvalid.shape,'yvalid',yvalid.shape,'non-zero:',np.count_nonzero(yvalid))

2022-05-26 13:04:26.134025
2022-05-26 13:04:26.179973
Xtrain (450, 5272) ytrain (450,) non-zero: 376
Xvalid (150, 5272) yvalid (150,) non-zero: 124


In [11]:
# RandomForestClassifier can only track feature names of type string.
num_problems=0
for name in Xtrain.columns:
    if not isinstance(name,str):
        num_problems += 1
        print(type(name),name)
if num_problems==0:
    print("Ok")

Ok


In [12]:
print(datetime.datetime.now())
class RF_Util:
    def __init__(self):
        self.model=RandomForestClassifier()
    def get_model(self):
        return self.model
    def set_train(self,X,y):
        self.Xtr = X
        self.ytr = y
    def set_validation(self,X,y):
        self.Xval = X
        self.yval = y
    def fit(self):
        self.model.fit(self.Xtr,self.ytr)
        #print(dir(self.model))  # see whether feature_names_in_ got created
    def validation_accuracy(self):
        ypred = self.model.predict(self.Xval)
        matches = np.count_nonzero(self.yval==ypred)
        accuracy = 100.0 * matches / len(ypred)  # bug fix
        return accuracy
    def validation_confusion(self):
        ypred = self.model.predict(self.Xval)
        cm = confusion_matrix(self.yval, ypred)
        return cm
    def important_features(self):
        names = self.model.feature_names_in_
        importances = self.model.feature_importances_
        pairs = np.column_stack( (names,importances) )
        top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
        # There must be a way to do this witout a loop!
        top_list = []
        for i in top_array:
             top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
        top_df = pd.DataFrame(top_list)
        return top_df

2022-05-26 13:04:26.292491


In [13]:
NUM_FEATURES=100
print('Original shape',Xtrain.shape,ytrain.shape)
print('Select %d features'%NUM_FEATURES)
print(datetime.datetime.now())
selector = SelectKBest(mutual_info_classif, k=NUM_FEATURES)
selector.fit_transform(Xtrain, ytrain)
print(datetime.datetime.now())

Original shape (450, 5272) (450,)
Select 100 features
2022-05-26 13:04:26.343482
2022-05-26 13:05:08.351103


In [14]:
num_features=len(selector.feature_names_in_)
scored_features=[]
for i in range(num_features):
    scored_features.append( (selector.scores_[i],selector.feature_names_in_[i]) )
best_features=[]
for t in sorted(scored_features,reverse=True):
    best_features.append(t[1])
    if len(best_features)>=NUM_FEATURES:
        break
Xtrain2 = Xtrain[ best_features ]
Xvalid2 = Xvalid[ best_features ]

In [15]:
print('Train on all Features')
rf2 = RF_Util()
rf2.set_train(Xtrain2,ytrain)
rf2.set_validation(Xvalid2,yvalid)
print(datetime.datetime.now())
rf2.fit()
print(datetime.datetime.now())
print('Accuracy:',rf2.validation_accuracy())
print('Confusion:')
print(rf2.validation_confusion())
print('The impurity-based feature importances.')
top = rf2.important_features()
top.head()

Train on all Features
2022-05-26 13:05:08.402249
2022-05-26 13:05:08.741401
Accuracy: 76.0
Confusion:
[[25  0  0  1  0  0]
 [ 0 24  1  6  0  0]
 [ 0  8 13  8  0  0]
 [ 0  5  1 14  0  3]
 [ 0  0  0  0 20  0]
 [ 0  0  1  0  2 18]]
The impurity-based feature importances.


,0,1
0,0.078581,Nuc_Intensity_MaxIntensity_Hematoxylin_max
1,0.062531,Nuc_Granularity_1_Hematoxylin_50%
2,0.050817,Nuc_Granularity_1_Hematoxylin_mean
3,0.032299,Nuc_Granularity_1_Hematoxylin_75%
4,0.023743,Nuc_Texture_SumAverage_Eosin_7_01_256_75%
